In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from datetime import date

In [35]:
maps = {47 : "Ancient",
       29 : "Cache",
       39 : "Cobblestone", 
       31 : "Dust2", 
       33 : "Inferno",
       32 : "Mirage",
       34 : "Nuke",
       40 : "Overpass",
       30 : "Seasons", 
       35 : "Train", 
       46 : "Vertigo"}

In [36]:
main_url = "https://www.hltv.org/stats/teams"

In [37]:
def get_table(url):
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    table = soup.find("table")
    
    return table

In [38]:
def correct_format(string):
    
    string = string.split()
    string = "-".join(string)
    
    return string.lower()

Getting most popular teams id and names

In [39]:
teams_table = get_table(main_url)
teams = dict()

for i, row in enumerate(teams_table.findAll("tr")):
    
    if i == 0: continue
    for j, elem in enumerate(row.findAll("td")):
        
        if j == 0:
            
            href = str(elem.find("a").get("href"))
            href = href.split("/")
            team_id = href[-2]
            
            if team_id not in teams.keys():
                
                team_name = correct_format(href[-1])
                teams[team_id] = team_name


In [40]:
teams

{'4863': 'tyloo',
 '6211': 'renegades',
 '8668': 'order',
 '7606': 'vici',
 '9565': 'vitality',
 '5929': 'space-soldiers',
 '9215': 'mibr',
 '8297': 'furia',
 '10577': 'sinners',
 '5973': 'liquid',
 '6667': 'faze',
 '6665': 'astralis',
 '5310': 'hellraisers',
 '7533': 'north',
 '7532': 'big',
 '4991': 'fnatic',
 '4411': 'ninjas-in-pyjamas',
 '4494': 'mouz',
 '5995': 'g2',
 '5005': 'complexity',
 '8135': 'forze',
 '4773': 'pain',
 '5422': 'dignitas',
 '6134': 'kinguin',
 '6290': 'luminosity',
 '5988': 'flipsid3',
 '5752': 'cloud9',
 '6947': 'teamone',
 '6010': 'chiefs',
 '9928': 'gamerlegion',
 '8637': 'sprout',
 '4869': 'ence',
 '4608': 'natus-vincere',
 '6902': 'godsent',
 '6137': 'sk',
 '7234': 'endpoint',
 '7865': 'havu',
 '6978': 'singularity',
 '5378': 'virtuspro',
 '6615': 'optic',
 '5991': 'envy',
 '7175': 'heroic',
 '4688': 'epsilon',
 '10386': 'skade',
 '7969': 'nemiga',
 '7718': 'movistar-riders',
 '7020': 'spirit',
 '6673': 'nrg',
 '7653': 'isurus',
 '5395': 'penta',
 '8068'

Getting teams lineups

In [41]:
months = {"Jan" : 1,
          "Feb" : 2,
          "Mar" : 3, 
          "Apr" : 4,
          "May" : 5,
          "Jun" : 6,
          "Jul" : 7,
          "Aug" : 8,
          "Sep" : 9, 
          "Oct" : 10, 
          "Nov" : 11,
          "Dec" : 12}

In [42]:
def convert_hltv_date(hltv_date):
    
    hltv_date = hltv_date.split()
    month = months[hltv_date[0]]
    year = int(hltv_date[1])

    return date(year, month, 1)

In [43]:
class roster():
    def __init__(self, lineup, start_date, end_date):
        self.lineup = lineup
        self.start_date = start_date
        self.end_date = end_date
    def get_lineup(self):
        return self.lineup
    def get_dates(self):
        return self.start_date, self.end_date

In [44]:
rosters = dict()


for team_id, team_name in tqdm(teams.items()):
    
    roster_url = f'{main_url}/lineups/{team_id}/{team_name}'
    html_text = requests.get(roster_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    
    for elem in soup.findAll("div", class_ = "lineup-container"):

        raw_lineup = elem.findAll("div", class_ = "text-ellipsis")
        
        lineup = []
        
        for i in raw_lineup:
            
            lineup.append(i.text)
            


        lineup_date = elem.findAll("div", class_ = "lineup-year")
        
        for month in lineup_date:

            dates = month.findAll("span")[:-1]

            if len(dates) == 1:

                start_date = convert_hltv_date(dates[0].text)
                end_date = date.today()

            else:

                start_date = convert_hltv_date(dates[0].text)
                end_date = convert_hltv_date(dates[1].text)

        if team_name not in rosters.keys(): rosters[team_name] = [roster(lineup, start_date, end_date)]

        else: rosters[team_name].append(roster(lineup, start_date, end_date))

100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:51<00:00,  1.20it/s]


In [45]:
rosters

{'tyloo': [<__main__.roster at 0x21c0ff1e310>,
 'renegades': [<__main__.roster at 0x21c0ff735b0>,
 'order': [<__main__.roster at 0x21c11485460>,
 'vici': [<__main__.roster at 0x21c113427c0>,
 'vitality': [<__main__.roster at 0x21c0ffd09d0>,
 'space-soldiers': [<__main__.roster at 0x21c0fad8e20>,
 'mibr': [<__main__.roster at 0x21c10c94610>,
 'furia': [<__main__.roster at 0x21c0f7825e0>,
 'sinners': [<__main__.roster at 0x21c10346820>,
 'liquid': [<__main__.roster at 0x21c0ff4ba90>,
 'faze': [<__main__.roster at 0x21c101b1c70>,
 'astralis': [<__main__.roster at 0x21c1084d610>,
 'hellraisers': [<__main__.roster at 0x21c115b5c40>,
 'north': [<__main__.roster at 0x21c1173fbe0>,
 'big': [<__main__.roster at 0x21c116b3040>,
 'fnatic': [<__main__.roster at 0x21c1022d2e0>,
 'ninjas-in-pyjamas': [<__main__.roster at 0x21c11662fd0>,
 'mouz': [<__main__.roster at 0x21c0fd7ad60>,
 'g2': [<__main__.roster at 0x21c1148dee0>,
 'complexity': [<__main__.roster at 0x21c1020a8b0>,
 'forze': [<__main__.ro

Getting final stats for all time

In [46]:
def str_to_date(string):
    
    day, month, year = map(int, string.split("/"))
    
    year += 2000
    
    return date(year, month, day)

In [47]:
def get_stats_from_table(table, map_title):
    
    stats = []
    columns = {0 : "date", 
           1 : "opponent", 
           2 : "event", 
           3 : "result"}
    
    for i, row in enumerate(table.findAll("tr")):
        
            if i == 0: continue
                
            row_dict = dict()
            row_dict["team_1"] = team_name
            
            for j, elem in enumerate(row.findAll("td")):
                
                if columns[j] == "result":
                    
                    score_1 = int(elem.text.split()[0])
                    score_2 = int(elem.text.split()[-1])
                    
                    row_dict["team_1_score"] = score_1
                    row_dict["team_2_score"] = score_2
                    
                elif columns[j] == "opponent": row_dict["team_2"] = correct_format(elem.text)
                   
                elif columns[j] == "date": row_dict["date"] = str_to_date(elem.text)
                else: row_dict[columns[j]] = elem.text
                    
                row_dict["map"] = map_title
                
            stats.append(row_dict)
            
    return stats

In [51]:
stats = []

for team_id, team_name in tqdm(teams.items()):
    for map_id, map_title in maps.items():
        
        current_url = f'{main_url}/map/{map_id}/{team_id}/{team_name}'
        print(current_url)
        current_table = get_table(current_url)
        stats += get_stats_from_table(current_table, map_title)
        

  0%|                                                                                           | 0/62 [00:00<?, ?it/s]

https://www.hltv.org/stats/teams//map/47/4863/tyloo
https://www.hltv.org/stats/teams//map/29/4863/tyloo
https://www.hltv.org/stats/teams//map/39/4863/tyloo
https://www.hltv.org/stats/teams//map/31/4863/tyloo
https://www.hltv.org/stats/teams//map/33/4863/tyloo
https://www.hltv.org/stats/teams//map/32/4863/tyloo
https://www.hltv.org/stats/teams//map/34/4863/tyloo
https://www.hltv.org/stats/teams//map/40/4863/tyloo
https://www.hltv.org/stats/teams//map/30/4863/tyloo
https://www.hltv.org/stats/teams//map/35/4863/tyloo
https://www.hltv.org/stats/teams//map/46/4863/tyloo


  2%|█▎                                                                                 | 1/62 [00:08<08:42,  8.56s/it]

https://www.hltv.org/stats/teams//map/47/6211/renegades
https://www.hltv.org/stats/teams//map/29/6211/renegades
https://www.hltv.org/stats/teams//map/39/6211/renegades
https://www.hltv.org/stats/teams//map/31/6211/renegades
https://www.hltv.org/stats/teams//map/33/6211/renegades
https://www.hltv.org/stats/teams//map/32/6211/renegades
https://www.hltv.org/stats/teams//map/34/6211/renegades
https://www.hltv.org/stats/teams//map/40/6211/renegades
https://www.hltv.org/stats/teams//map/30/6211/renegades
https://www.hltv.org/stats/teams//map/35/6211/renegades
https://www.hltv.org/stats/teams//map/46/6211/renegades


  3%|██▋                                                                                | 2/62 [00:16<08:07,  8.12s/it]

...

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 60/62 [09:12<00:18,  9.24s/it]

https://www.hltv.org/stats/teams//map/47/4501/alternate-attax
https://www.hltv.org/stats/teams//map/29/4501/alternate-attax
https://www.hltv.org/stats/teams//map/39/4501/alternate-attax
https://www.hltv.org/stats/teams//map/31/4501/alternate-attax
https://www.hltv.org/stats/teams//map/33/4501/alternate-attax
https://www.hltv.org/stats/teams//map/32/4501/alternate-attax
https://www.hltv.org/stats/teams//map/34/4501/alternate-attax
https://www.hltv.org/stats/teams//map/40/4501/alternate-attax
https://www.hltv.org/stats/teams//map/30/4501/alternate-attax
https://www.hltv.org/stats/teams//map/35/4501/alternate-attax
https://www.hltv.org/stats/teams//map/46/4501/alternate-attax


 98%|████████████████████████████████████████████████████████████████████████████████▋ | 61/62 [09:23<00:09,  9.71s/it]

https://www.hltv.org/stats/teams//map/47/7681/izako-boars
https://www.hltv.org/stats/teams//map/29/7681/izako-boars
https://www.hltv.org/stats/teams//map/39/7681/izako-boars
https://www.hltv.org/stats/teams//map/31/7681/izako-boars
https://www.hltv.org/stats/teams//map/33/7681/izako-boars
https://www.hltv.org/stats/teams//map/32/7681/izako-boars
https://www.hltv.org/stats/teams//map/34/7681/izako-boars
https://www.hltv.org/stats/teams//map/40/7681/izako-boars
https://www.hltv.org/stats/teams//map/30/7681/izako-boars
https://www.hltv.org/stats/teams//map/35/7681/izako-boars
https://www.hltv.org/stats/teams//map/46/7681/izako-boars


100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [09:31<00:00,  9.22s/it]


In [52]:
def current_lineup(team, date):
    
    if team in rosters.keys():
        for roster in rosters[team]:
            dates = roster.get_dates()


            if date >= dates[0] and date < dates[1]:
                lineup = roster.get_lineup()
                return lineup
    return None

In [53]:
for game in stats:
    
    team_1 = game["team_1"]
    team_2 = game["team_2"]
    date = game["date"]
    
    lineup_1 = current_lineup(team_1, date)
    lineup_2 = current_lineup(team_2, date)
    
    
    game["lineup_1"] = lineup_1
    game["lineup_2"] = lineup_2

In [54]:
df = pd.DataFrame.from_dict(stats)

In [55]:
df = df.dropna()
df.to_csv("stats.csv")